<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Elyza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab で ELYZA-japanese-Llama-2-7b を試す
https://note.com/npaka/n/nbb94b45f47a5

## パッケージのインストール

In [ ]:
!pip install transformers accelerate bitsandbytes

## トークナイザーとモデルの準備
今回は、高速な指示モデル (elyza/ELYZA-japanese-Llama-2-7b-fast-instruct) を利用しています。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct"
)
model = AutoModelForCausalLM.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

### 推論の実行

In [ ]:
# プロンプトの準備
prompt = """<s>[INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

まどか☆マギカでは誰が一番かわいい？ [/INST]"""

# 推論の実行
with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
print(output)

### プロンプトの書き方

プロンプトの書式は、「Llama 2」と同様です。

```
 <s>[INST] <<SYS>>
 {システムメッセージ}
 <</SYS>>

 {ユーザーメッセージ} [/INST]
```

## ストリーミング出力

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread

# ストリーミング出力
with torch.no_grad():
    streamer = TextIteratorStreamer(tokenizer)
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    generation_kwargs = dict(
        input_ids=token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

# 出力
for new_text in streamer:
    print(new_text.replace(" ", ""), end="")